# Activity 6: Classifying an Image

In this activity, we'll use a convolutional neural network (CNN) to classify a real image -- or maybe a few of them.

We're going to use Inception v3, the same CNN used by Esteva et al. in the paper you read.

This activity will force us to think through a few important details, including:
- how do we load images in Python?
- how do we get an image (or batch of images) ready to be passed through a CNN?
- what does the output of the CNN look like, and how do we interpret it?

We'll begin with our usual imports plus a few new ones: we'll be importing `tensorflow` as well as `tensorflow_hub`, which will help us load an Inception v3 model that has already been trained (i.e. *pre*-trained) on ImageNet.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

Let's begin by using `tensorflow_hub` to load Inception v3. I found this URL by searching available models on [TensorFlow Hub](https://tfhub.dev). This step may take a minute or two. Once it's complete, we'll have the pre-trained Inception v3 model as a [SavedModel object](https://www.tensorflow.org/guide/saved_model) called `inception_v3`.

In [2]:
INCEPTION_V3_URL = "https://tfhub.dev/google/imagenet/inception_v3/classification/4"
inception_v3 = hub.load(INCEPTION_V3_URL)

Before we actually do anything with our model, let's make sure we understand the problem it's designed to solve. This model takes an image as input, and produces a vector of length 1001 as its output. The 1001 elements of this vector correspond to the 1001 different image classes (i.e., labels) in ImageNet. Let's use `pandas` to load these labels from URL and take a look at them.

In [ ]:
LABELS_URL = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"
df = pd.read_table(LABELS_URL, header=None)
df.columns = ['label']
df

By default, `pandas` shows us the first few labels and the last few. We can see that Inception v3 knows how to identify everything from *great white shark* to *toilet paper*. Note that label 0, *background*, indicates the absence of a recognized image class. We'll set these labels aside for now but return to them after we've applied Inception v3 to a first image.

Now let's go ahead and load an image. The code in the next block isn't important; we're just saving a picture of a tiger in our current working directory.

In [ ]:
import urllib.request

TIGER_IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/b/b0/Bengal_tiger_%28Panthera_tigris_tigris%29_female_3_crop.jpg"

urllib.request.urlretrieve(TIGER_IMAGE_URL, 'tiger_image.jpg')

We can now load this image as a `numpy` array using the `cv2` library, which is a little bit easier to work with than the standard Python image library (`PIL`). One quirk of `cv2` is that images are loaded in BGR format rather than RGB. To correct this, we'll reverse the order of the last axis in our array with `tiger_image_bgr[:, :, ::-1]`. We can then use `plt.imshow` to display the image.

In [ ]:
import cv2

tiger_image_bgr = cv2.imread('tiger_image.jpg')
tiger_image_rgb = tiger_image_bgr[:, :, ::-1]
plt.imshow(tiger_image_rgb)
plt.show()

A few more steps are required get this image into a format Inception v3 knows how to interpret. First, Inception v3 expects RGB images (check!) that are 299 by 299 pixels in size. So, let's go ahead and resize our image with `cv2.resize`. Note that the original resolution of this image, along with most other images we're used to seeing, was much higher than 299 by 299.

In [ ]:
tiger_image_resized = cv2.resize(tiger_image_rgb, (299, 299))
plt.imshow(tiger_image_resized)
plt.show()

Three final, minor steps are needed:
1. Add a fourth axis to our image. This is needed because Inception v3 is expecting a *batch* of images, not just a single image. After adding the new axis, we can think of our array of a batch consisting of just one image. However, we could process multiple images at a time by appending more images along this axis.
2. Inception v3 expects pixel values to range from 0 to 1, but our pixels range from 0 to 255. We can fix this by dividing our array by 255.
3. To use our Tensorflow model, we'll need to convert our array to a Tensorflow tensor. This is just the way Tensorflow works.

In [7]:
tiger_image_batch = tiger_image_resized[np.newaxis, :, :, :]
tiger_image_rescaled = tiger_image_batch / 255
tiger_tensor = tf.convert_to_tensor(tiger_image_rescaled, dtype=tf.float32)

Our tiger image is now ready for modeling. Let's print it out to help us understand the format a bit better:

In [ ]:
print(tiger_tensor)

Now that our image is ready, we can apply Inception v3 in just one line! This will return another Tensorflow tensor, which we can convert to a `numpy` array using the tensor's `.numpy()` method.

In [9]:
tiger_logits = inception_v3(tiger_tensor).numpy()

Inception v3 returns predictions as logits, but if needed, we can convert these to probabilities using the softmax function. For now, it's not needed. Instead, let's add the predictions to our dataframe of labels so we can make sense of the predictions in terms of the object classes to which they correspond.

Note that to do this, we'll have to `.squeeze()` our array to remove the leading, *batch* axis. If we'd been processing multiple images rather than just one, Inception would've made predictions for each image and stacked them along this leading axis.

In [10]:
df['tiger_logits'] = tiger_logits.squeeze()

We can now sort our dataframe by the predicted logit values, in descending order, and see whether Inception v3 got it right!

In [ ]:
df.sort_values(by='tiger_logits', ascending=False).head(20)

With the remaining time, please try this out on another image, or better yet, a batch of several images. To do this, note that you can upload images directly from your local computer using the Colab file browser, which can be accessed from the panel on the left.